In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.ccs,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
# from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema, 
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [8]:
db[User].values()


[
    User({'_id': '835f8fe6-b437-488f-91de-9193120a6315', 'name': 'John', 'age': 31}),
    User({'_id': 'f20c15b5-6e46-4660-b0a9-a5aab55fad62', 'name': 'Jane', 'age': 56}),
    User({'_id': '6fa4a92e-7c1f-4c9f-b8cb-2b01764eb10d', 'name': 'Jack', 'age': 66})
]

In [9]:
db[Search.term].values()

['test']

In [10]:
list(db[Search.result_count].items())

[('test', 3)]

In [11]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 2, '1d61ea2d-a3f4-472d-80e8-ccd40c8fae72'),
        Organization({'_id': '1d61ea2d-a3f4-472d-80e8-ccd40c8fae72', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', 1, '6344211d-c6fd-4104-a32d-8ebe92a3d306'),
        Organization({'_id': '6344211d-c6fd-4104-a32d-8ebe92a3d306', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    ),
    (
        ('test', 0, '23dabc23-2076-43c5-af8f-a7f60a1e062b'),
        Organization({'_id': '23dabc23-2076-43c5-af8f-a7f60a1e062b', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    )
]

In [12]:
list(db[Search.results].items())[0][1].start.year

2020

In [13]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [14]:
db[Search.results].df()

schema_Search_term,number,name,start,end,status,org__id
str,i64,str,date,date,str,str
"""test""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""1d61ea2d-a3f4-472d-80e8-ccd40c…"
"""test""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""6344211d-c6fd-4104-a32d-8ebe92…"
"""test""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""23dabc23-2076-43c5-af8f-a7f60a…"


In [15]:
db[Search.results.x.org].df()

schema_Search_term,schema_Search_results_number,_id,name,address,city
str,i64,str,str,str,str
"""test""",2,"""1d61ea2d-a3f4-472d-80e8-ccd40c…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""test""",1,"""6344211d-c6fd-4104-a32d-8ebe92…","""Car Shop""","""Main Street 3""","""Cartown"""
"""test""",0,"""23dabc23-2076-43c5-af8f-a7f60a…","""Bakery""","""Main Street 1""","""Bakerville"""


In [16]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""23dabc23-2076-43c5-af8f-a7f60a…","""Bakery""","""Main Street 1""","""Bakerville"""
"""1d61ea2d-a3f4-472d-80e8-ccd40c…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""6344211d-c6fd-4104-a32d-8ebe92…","""Car Shop""","""Main Street 3""","""Cartown"""


In [17]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev148+g44d4d92.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '0b60aff2311656cb53174ff572cf2a864632b001',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.Organization': 3,
            'schema.Project': 3,
            'schema.Membership': 7,
            'schema.User': 3,
            'schema.Task': 6,
            'schema.SearchResult': 3,
            'schema.Search': 1
        },
        'arrays': {'schema_Organization_ccs': 7},
        'relations': {'schema.Membership': 7, 'schema.SearchResult': 3}
    },
    'backend': None
}

In [18]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,score,_from_term,_to_number
str,f64,str,i64
"""4376575091""",0.9,"""test""",0


In [19]:
db[Organization.ccs].df()

schema_Organization__id,schema_Organization_ccs_idx,value
str,i64,str
"""23dabc23-2076-43c5-af8f-a7f60a…",0,"""USA"""
"""23dabc23-2076-43c5-af8f-a7f60a…",1,"""Mexico"""
"""23dabc23-2076-43c5-af8f-a7f60a…",2,"""Canada"""
"""6344211d-c6fd-4104-a32d-8ebe92…",0,"""USA"""
"""6344211d-c6fd-4104-a32d-8ebe92…",1,"""Canada"""
"""1d61ea2d-a3f4-472d-80e8-ccd40c…",0,"""USA"""
"""1d61ea2d-a3f4-472d-80e8-ccd40c…",1,"""Mexico"""


In [20]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         440818 function calls (408914 primitive calls) in 0.782 seconds

   Ordered by: cumulative time
   List reduced from 2287 to 156 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52    0.000 

   0.000    0.274    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:697(_load_rec_from_item)
       52    0.002    0.000    0.272    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:572(_load_record)
       56    0.000    0.000    0.266    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:838(load)
   168/57    0.001    0.000    0.254    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:665(_load_records)
       19    0.000    0.000    0.226    0.012 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1473(__ior__)
       19    0.001    0.000    0.226    0.012 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2935(_mutate)
14044/6865    0.016    0.000    0.181    0.000 C:\Users\lworm\Repos\py-research\src\py_research\caching.py:382(__get__)
       17    0.000    0.000    0.176    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3072(_mutate_f